## ZX simulation validation

Performs a set of efficiency, acceptance, and resolution plots based on the ZX toy MC samples

The notebook runs with the 96, 97 (python3 preferentially) software stack in SWAN

In [ ]:
import ROOT
import itertools
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# plot configurations
ROOT.gStyle.SetOptTitle(0)
ROOT.gStyle.SetOptStat(0)
ROOT.gStyle.SetTextFont(42)
ROOT.gStyle.SetPadBottomMargin(0.1)
ROOT.gStyle.SetPadLeftMargin(0.12)
ROOT.gStyle.SetPadTopMargin(0.05)
ROOT.gStyle.SetPadRightMargin(0.03)

outdir='/eos/user/p/psilva/www/ExclusiveAna_2017_unblind_multi/signal'

#prepare output directory
import os
os.system('mkdir -p %s'%outdir)

#available info
masses=[600,800,1000,1200,1400,1600]
sides=['Pos','Neg']
titles={'multi':'multiRP','pix':'Pixels','strip':'Strips'}
ms={'multi':20,'pix':26,'strip':32}
algos=titles.keys()

vtitles={'mpp':'m(pp) [GeV]',
         'pzpp':'$p_{z}$(pp) [GeV]',
        }
vlist=vtitles.keys()
effbins={'mpp':np.linspace(0,2000,100),'pzpp':np.linspace(-2500,2500,100),'xi':np.linspace(0,0.2,100)}

fillGraphs=True

# read the data to memory
rdf = ROOT.RDataFrame("tuple", "/eos/cms/store/cmst3/group/top/PPSZX/ZX_validation.root")
df=pd.DataFrame(rdf.AsNumpy())
df=df.rename(columns={"gen_pzpp": "pzpp_gen"})
df.head()

### Auxiliary functions

In [ ]:
def getHistoWithErrors(x,w,hbins):
    
    """ computes a weighted histogram with associated errors using error propagation """
    
    bins = np.digitize(x,hbins)    

    #project entries in each bin summing up the weights
    h=[]
    for i in range(len(hbins)):
        ni  = w[np.where(bins==i+1)[0]]
        h.append( [np.sum(ni),np.sqrt(np.sum(ni**2.))] )
    
    return np.array(h)

def getEfficiency(df,baseCut,passCut,vname,wname,hbins,debug=False):
    
    """ compute the ratio of pass/total histograms with error propagation """   
    
    df_den = df[baseCut]
    h_den  = getHistoWithErrors(x=df_den[vname].to_numpy(),
                                w=df_den[wname].to_numpy(),
                                hbins=hbins)
    df_num = df_den[passCut]
    h_num  = getHistoWithErrors(x=df_num[vname].to_numpy(),
                                w=df_num[wname].to_numpy(),
                                hbins=hbins)    
    
    #regularize the numbers to become close to 1
    regVec=-np.floor(np.log(h_den[:,0]),where=h_den[:,0]!=0)
    regVec=np.exp(regVec,where=regVec!=np.nan,out=np.ones_like(regVec))
    h_den=(h_den.T*regVec).T
    h_num=(h_num.T*regVec).T
    
    d,ed=h_den[:,0],h_den[:,1]
    n,en=h_num[:,0],h_num[:,1]
    eff=np.true_divide(n.astype(np.double), 
                       d.astype(np.double), 
                       out=np.zeros_like(n), 
                       where=d!=0).astype(np.double)
    
    eeffn = np.sqrt( (d*en)**2 + (n*ed)**2 )
    eeffd = d*d
    eeff  = np.true_divide(eeffn.astype(np.double), 
                           eeffd.astype(np.double), 
                           out=np.zeros_like(eeffn), 
                           where=eeffd!=0,
                           dtype=np.double)
    
    eff   = np.stack((eff, eeff), axis=-1)
    
    if debug:
        print('Histogram and efficency dump')
        print(np.stack((h_den,h_num,eff),axis=-1))
        
    return eff


def showEfficiencies(x,effList,titleList=None,loc='best',xtitle=None,ytitle='Acceptance',extraText=[],outpname=None,grid=True,ylim=(0,1),exts=['png','pdf']):
    
    """ a wrapper to show the efficiencies """
    
    #prepare figure
    fig, ax = plt.subplots()
    fig.set_size_inches(6, 6)
    ax.set_xlabel(xtitle,fontsize=14)
    ax.tick_params(axis='x', labelsize=14)
    ax.set_ylabel(ytitle,fontsize=14)
    ax.yaxis.label.set_size(18)
    ax.tick_params(axis='y', labelsize=14)
    
    #show the efficiency graphs
    for i in range(len(effList)):
        eff=effList[i]
        kwargs={'yerr':eff[:,1]}
        if titleList: kwargs['label']=titleList[i]
        plt.errorbar(x,eff[:,0],fmt='o',**kwargs)
        
    #extra text
    ax.text(0,   1.02,'CMS',                       transform=ax.transAxes, fontsize=16, weight='bold')
    ax.text(0.12,1.02,'Simulation Preliminary',    transform=ax.transAxes, fontsize=15, style='italic')
    for i in range(len(extraText)):
        ax.text(0.05,0.95-i*0.03, extraText[i], transform=ax.transAxes, fontsize=14, style='italic')
       
    plt.legend(loc=loc,fontsize=14)
    if grid: plt.grid(True)
    plt.ylim(*ylim)
    plt.tight_layout()
    
    if outpname:
        for ext in exts:
            plt.savefig(outpname+'.'+ext)
    plt.show()

In [ ]:
# test
_=getEfficiency(df,
                baseCut=((df['mX']==1000) & (df['csiPos_gen']>0)),
                passCut=(df['csiPos_multi']>0),
                vname='csiPos_gen',
                wname='wgt',
                hbins=np.linspace(0,0.2,50),
                debug=True)        

### Single RP efficiency curves, as function of the generator level xi variable

In [ ]:
def fillSingleRPEffs(hbins=effbins['xi']):
    
    """fill the efficiency graphs (just needs to be done once, as it'll store the results in a pickle file)"""
    
    effGrs={}

    keys=list(itertools.product(masses,sides,algos))
    for k in keys:
        mX,side,algo=k
        effGrs[k] = getEfficiency(df,
                                  baseCut=((df['mX']==mX) & (df['csi%s_gen'%side]>0)),
                                  passCut=(df['csi%s_%s'%(side,algo)]>0),
                                  vname='csi{0}_gen'.format(side),
                                  wname='wgt',
                                  hbins=hbins)        
        
    pckF=os.path.join(outdir,'effgrs.pck')
    with open(pckF,'wb') as fout:
        pickle.dump(hbins,fout,pickle.HIGHEST_PROTOCOL)
        pickle.dump(effGrs,fout,pickle.HIGHEST_PROTOCOL)
    print('Results saved in',pckF)

if fillGraphs: fillSingleRPEffs()

In [ ]:
# display plots (reads always from the pickle file)

with open(os.path.join(outdir,'effgrs.pck'),'rb') as fin:
    x=pickle.load(fin)
    effGrs=pickle.load(fin)
    
keys=list(itertools.product(masses,sides))
for k in keys:
    
    mX,side=k
    
    effList=[]
    titleList=[]
    for a in ['pix','strip','multi']:
        grk=(mX,side,a)
        effList.append(effGrs[grk])
        titleList.append(titles[a])        
        
    showEfficiencies(x,
                     effList,
                     titleList,
                     xtitle=r'Generated $\xi_{%s}$'%('+' if side=='Pos' else '-'),
                     extraText=[r'$m_{X}=%d GeV$'%mX],
                     outpname=os.path.join(outdir,'rpeff_{0}_{1}'.format(mX,side)),
                     ylim=(0.1,1.1)
                    )    


## Combined PPS efficiency plots

In [ ]:
def fillCombinedRPEffs():
    
    """ fill the combined PPS efficiency graphs (just needs to be done once, as it'll store the results in a pickle file) """

    combineEffGrs={}
    
    keys=list(itertools.product(masses,algos,vlist))
    for k in keys:
        mX,algo,v=k    
    
        combineEffGrs[k] = getEfficiency(df,
                                         baseCut=((df['mX']==mX) & (df['csiPos_gen']>0) & (df['csiNeg_gen']>0)),
                                         passCut=(df['mpp_%s'%algo]>0),
                                         vname='{0}_gen'.format(v),
                                         wname='wgt',
                                         hbins=effbins[v])
        
    pckF=os.path.join(outdir,'combeffgrs.pck')
    with open(pckF,'wb') as fout:
        pickle.dump(effbins,fout,pickle.HIGHEST_PROTOCOL)
        pickle.dump(combineEffGrs,fout,pickle.HIGHEST_PROTOCOL)
    
    print('Results saved in',pckF)

if fillGraphs: fillCombinedRPEffs()     

In [ ]:
with open(os.path.join(outdir,'combeffgrs.pck'),'rb') as fin:
    x=pickle.load(fin)
    effGrs=pickle.load(fin)
    
keys=list(itertools.product(masses,vlist))
for k in keys:
    
    mX,v=k
    
    effList=[]
    titleList=[]
    for a in ['pix','strip','multi']:
        grk=(mX,a,v)
        effList.append(effGrs[grk])
        titleList.append(titles[a])        
        
    showEfficiencies(effbins[v],
                     effList,
                     titleList,
                     xtitle=r'Generated %s'%vtitles[v],
                     extraText=[r'$m_{X}=%d GeV$'%mX],
                     outpname=os.path.join(outdir,'combrpeff_{0}_{1}'.format(mX,v)),
                     ylim=(0.1,1.1),
                     loc='lower left'
                    )

## Resolution plots

In [ ]:
def fillResolutions():
    
    """ fill the combined PPS efficiency graphs (just needs to be done once, as it'll store the results in a pickle file) """

    resolSummary=[]
    
    for a in algos:
        
        #compute resolutions per algo
        resolSummary.append( df[['mX','csiPos_%s'%a,'csiNeg_%s'%a]].copy() )
        resolSummary[-1]['algo']      = a
        resolSummary[-1]['resCsiPos'] = df['csiPos_%s'%a]-df['csiPos_gen']
        resolSummary[-1]['resCsiNeg'] = df['csiNeg_%s'%a]-df['csiNeg_gen']
        resolSummary[-1]['resMpp']    = df['mpp_%s'%a]/df['mpp_gen']-1
        resolSummary[-1]['resMmass']  = df['mmass_%s'%a]/df['mmass_gen']-1

        #require both arms in acceptance
        mask=(resolSummary[-1]['csiPos_%s'%a]>0) & (resolSummary[-1]['csiNeg_%s'%a]>0)
        resolSummary[-1]=resolSummary[-1][mask]
                
    return pd.concat(resolSummary, ignore_index=True)

resolSummary = fillResolutions()   

#show the summaries as box plots
for var,varTitle in [('resCsiPos', r'$\xi_{rec}(+)-\xi_{gen}(+)$'),
                     ('resCsiNeg', r'$\xi_{rec}(-)-\xi_{gen}(-)$'),
                     ('resMpp',    r'$m_{rec}(pp)/m_{gen}(pp)-1$'),
                     ('resMmass',  r'$m_{mass,rec}/m_{mass,gen}-1$'),
                    ]:
    fig, ax = plt.subplots()
    fig.set_size_inches(20, 6)
    g=sns.boxplot(x="mX",y=var, hue="algo",data=resolSummary, ax=ax, whis=np.inf)    
    ax.set_xlabel(r'm$_{X}$ [GeV]',fontsize=16)
    ax.xaxis.label.set_size(18)
    ax.set_ylabel(varTitle,fontsize=16)
    ax.yaxis.label.set_size(18)
    ax.text(0,   1.02,'CMS',                    transform=ax.transAxes, fontsize=16,weight='bold')
    ax.text(0.04,1.02,'Simulation Preliminary', transform=ax.transAxes, fontsize=15, style='italic')

    plt.grid(True)
    plt.tight_layout()

    for ext in ['png','pdf']:
        figName=os.path.join(outdir,'resol_{0}.{1}'.format(var,ext))
        plt.savefig( figName)
 
    plt.show()
        